In [ ]:
#https://drscotthawley.github.io/blog/2019/12/21/PCA-From-Scratch.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [2]:
N = 100 
x = np.random.normal(size=N)
y = 0.5*x + 0.2*(np.random.normal(size=N))

In [3]:
fig = go.Figure(data=[go.Scatter(x=x, y=y, mode='markers', 
                marker=dict(size=8,opacity=0.5), name="data" )])
fig.update_layout( xaxis_title="x", yaxis_title="y",
    yaxis = dict(scaleanchor = "x",scaleratio = 1) )
fig.show()

In [4]:
print("Variance in x =",np.var(x))
print("Variance in y =",np.var(y))

Variance in x = 1.0489413372264906
Variance in y = 0.3111129201697987


### Covariance 
You'll notice in the above graph that as $x$ varies, so does $y$ -- pretty much.  So $y$ is "*covariant*" with $x$.  ["Covariance indicates the level to which two variables vary together."](https://docs.scipy.org/doc/numpy/reference/generated/numpy.cov.html)  To compute it, it's kind of like the regular variance, except that instead of squaring the deviation from the mean for one variable, we multiply the deviations for the two variables:

$${\rm Cov}(x,y) = {1\over N-1}\sum_{j=1}^N (x_j-\mu_x)(y_j-\mu_j),$$
where $\mu_x$ and $\mu_y$ are the means for the x- and y- componenets of the data, respectively.  Note that you can reverse $x$ and $y$ and get the same result, and the covariance of a variable with itself is just the regular variance -- but with one caveat! 

The caveat is that we're dividing by $N-1$ instead of $N$, so unlike the regular variance we're not quite taking the mean.  Why this?  Well, for large datasets this makes essentially no difference, but for small numbers of data points, using $N$ can give values that tend to be a bit too small for most people's tastes, so the $N-1$ was introduced to "reduce small sample bias." 

In [5]:
def covariance(a,b):
    return ( (a - a.mean())*(b - b.mean()) ).sum() / (len(a)-1)

print("Covariance of x & y =",covariance(x,y))
print("Covariance of y & x =",covariance(x,y))
print("Covariance of x with itself =",covariance(x,x),", variance of x =",np.var(x))
print("Covariance of y with itself =",covariance(y,y),", variance of x =",np.var(y))

Covariance of x & y = 0.5481155996854675
Covariance of y & x = 0.5481155996854675
Covariance of x with itself = 1.0595367042691826 , variance of x = 1.0489413372264906
Covariance of y with itself = 0.31425547491898853 , variance of x = 0.3111129201697987


In [6]:
data = np.stack((x,y),axis=1)   # pack the x & y data together in one 2D array
print("data.shape =",data.shape)
cov = np.cov(data.T)   # .T b/c numpy wants varibles along rows rather than down columns?
print("covariance matrix =\n",cov)

data.shape = (100, 2)
covariance matrix =
 [[1.0595367  0.5481156 ]
 [0.5481156  0.31425547]]


In [7]:
z = -.5*x + 2*np.random.uniform(size=N)
data = np.stack((x,y,z)).T
print("data.shape =",data.shape)
cov = np.cov(data.T)
print("covariance matrix =\n",cov)

# Plot our data
import plotly.graph_objects as go
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,mode='markers', marker=dict(size=8,opacity=0.5), name="data" )])
fig.update_layout( xaxis_title="x", yaxis_title="y", yaxis = dict(scaleanchor = "x",scaleratio = 1) )
fig.show()

data.shape = (100, 3)
covariance matrix =
 [[ 1.0595367   0.5481156  -0.46878808]
 [ 0.5481156   0.31425547 -0.24227001]
 [-0.46878808 -0.24227001  0.50416514]]


To do Principal Component Analysis, we need to find the aforementioned "components," and this requires finding *eigenvectors* for our dataset's covariance matrix.

# What is an eigenvector, *really*?
First a **definition**.  (Stay with me!  We'll flesh this out in what comes after this.)

Given some matrix (or 'linear operator') ${\bf A}$ with dimensions $n\times n$ (i.e., $n$ rows and $n$ columns), there exist a set of $n$ vectors $\vec{v}_i$ (each with dimension $n$, and $i = 1...n$ counts which vector we're talking about) **such that** multiplying one of these vectors by ${\bf A}$ results in a vector (anti)parallel to $\vec{v}_i$, with a length that's multiplied by some constant $\lambda_i$.  In equation form:

$${\bf A}\vec{v}_i = \lambda_i \vec{v}_i,\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ (1)$$

where the constants $\lambda_i$ are called *eigenvalues* and the vectors $\vec{v}_i$ are called *eigenvectors*. 

<span style="text-align:right; font-style:italic; font-size:80%;">(Note that I'm departing a bit from common notation that uses $\vec{x}_i$ instead of $\vec{v}_i$; I don't want people to get confused when I want to use $x$'s for coordinate variables.)</span>

https://www.quora.com/What-does-Eigen-mean/answer/Daniel-McLaury

In [8]:
from numpy import linalg as LA
lambdas, vs = LA.eig(cov)
lambdas, vs

(array([1.60306293, 0.02400499, 0.2508894 ]),
 array([[-0.79738183, -0.47452591,  0.37283694],
        [-0.4204877 ,  0.88003084,  0.22076189],
        [ 0.43286524, -0.01925817,  0.90125291]]))

Ok sort of kidding; we'll do it "from scratch".  But, one caveat before we start: Some matrices can be "weird" or "problematic" and have things like "singular values."  There are  sophisticated numerical libraries for doing this, and joking aside, for real-world numerical applications you're better off calling a library routine that other very smart and very careful people have written for you.  But for now, we'll do the straightforward way which works pretty well for many cases.
We'll follow the basic two steps:


1.   Find the eigenvalues  
2.   'Plug in' each eigenvalue to get a system of linear equations for the values of the components of the corresponding eigenvector
3. Solve this linear system. 



## 1. Find the eigenvalues  
Ok I'm hoping you at least can recall what a [determinant](https://en.wikipedia.org/wiki/Determinant) of a matrix is. Many people, even if they don't know what a determinant is good for (e.g. tons of proofs & properties all rest on the determinant), they still at least remember how to calculate one. 

The way to get the eigenvalues is to take the determinant of the difference between a $\bf{A}$ and $\lambda$ times the *identity matrix* $\bf{I}$ (which is just ones along the diagonal and zeros otherwise) and set that difference equal to zero...

$$det( \bf{A} - \lambda I) = 0 $$

> <small>Just another observation: Since ${\bf I}$ is a square matrix, that means $\bf{A}$ has to be a square matrix too.</small>

Then solving for $\lambda$ will give you a *polynomial equation* in $\lambda$, the solutions to (or roots of) which are the eigenvectors $\lambda_i$. 

Let's do an example:

$$
{\bf A} = 
\begin{bmatrix}
-2 & 2 & 1\\
-5 & 5 & 1\\
-4 & 2 & 3
\end{bmatrix}
$$
To find the eigenvalues we set 
$$
det( \bf{A} - \lambda I) =
\begin{vmatrix}
-2-\lambda & 2 & 1\\
-5 & 5-\lambda & 1\\
-4 & 2 & 3-\lambda
\end{vmatrix} = 0.$$


This gives us the equation...
$$0 = \lambda^3 - 6\lambda^2 + \lambda - 6$$

which has the 3 solutions (in descending order)
$$ \lambda = 3, 2, 1.$$ 

<small>*(Aside: to create an integer matrix with integer eigenvalues, I used [this handy web tool](https://ericthewry.github.io/integer_matrices/))*.</small>

Just to check that against the numpy library:

In [9]:
A = np.array([[-2,2,1],[-5,5,1],[-4,2,3]])

def sorted_eig(A):  # For now we sort 'by convention'. For PCA the sorting is key. 
    lambdas, vs = LA.eig(A)
    # Next line just sorts values & vectors together in order of decreasing eigenvalues
    lambdas, vs = zip(*sorted(zip(list(lambdas), list(vs.T)),key=lambda x: x[0], reverse=True))
    return lambdas, np.array(vs).T  # un-doing the list-casting from the previous line

lambdas, vs = sorted_eig(A)
lambdas # hold off on printing out the eigenvectors until we do the next part!

(3.0000000000000036, 1.9999999999999993, 0.9999999999999991)

# 2. Use the eigenvalues to get the eigenvectors

Although it was anncounced in mid 2019 that [you can get eigenvectors directly from eigenvalues](https://arxiv.org/abs/1908.03795), the usual way people have done this for a very long time is to go back to the matrix $\bf{A}$ and solve the *linear system* of equation (1) above, for each of the eigenvalues.  For example, for $\lambda_1=-1$, we have 

$$
{\bf}A \vec{v}_1 = -\vec{v}_1
$$
i.e.

$$
\begin{bmatrix}
-2 & 2 & 1\\
-5 & 5 & 1\\
-4 & 2 & 3
\end{bmatrix}
\begin{bmatrix}
v_{1x}\\
v_{1y}\\
v_{1z}\\
\end{bmatrix}
= 
-\begin{bmatrix}
v_{1x}\\
v_{1y}\\
v_{1z}\\
\end{bmatrix}
$$
This amounts to 3 equations for 3 unknowns,...which I'm going to assume you can handle...  For the other eigenvalues things proceed similarly.  The solutions we get for the 3 eigenvalues are: 

$$\lambda_1 = 3: \ \ \ \vec{v}_1 = (1,2,1)^T$$
  

$$\lambda_2 = 2: \ \ \ \vec{v}_2 = (1,1,2)^T$$

$$\lambda_3 = 1: \ \ \ \vec{v}_3 = (1,1,1)^T$$


Since our original equation (1) allows us to scale eigenvectors by any artibrary constant, often we'll express eigenvectors as *unit* vectors $\hat{v}_i$.  This will amount to dividing by the length of each vector, i.e. in our example multiplying by $(1/\sqrt{6},1/\sqrt{6},1/\sqrt{3})$.  

In this setting 

$$\lambda_1 = 3: \ \ \ \hat{v}_1 = (1/\sqrt{6},2/\sqrt{6},1/\sqrt{6})^T$$
  
$$\lambda_2 = 2: \ \ \ \hat{v}_2 = (1/\sqrt{6},1/\sqrt{6},2/\sqrt{6})^T$$

$$\lambda_3 = 1: \ \ \ \hat{v}_3 = (1,1,1)^T/\sqrt{3}$$



Checking our answers (left) with numpy's answers (right):

In [ ]:
print(" "*15,"Ours"," "*28,"Numpy")
print(np.array([1,2,1])/np.sqrt(6), vs[:,0])
print(np.array([1,1,2])/np.sqrt(6), vs[:,1])
print(np.array([1,1,1])/np.sqrt(3), vs[:,2])

In [ ]:
print("A*v_1 / 3 = ",np.matmul(A, np.array([1,2,1]).T)/3 ) # Dividing by eigenvalue 
print("A*v_2 / 2 = ",np.matmul(A, np.array([1,1,2]).T)/2 ) #    to get vector back
print("A*v_3 / 1 = ",np.matmul(A, np.array([1,1,1]).T) )

In [ ]:
# Now that we know we can get the same answers as the numpy library, let's use it
lambdas, vs = sorted_eig(cov)  # Compute e'vals and e'vectors of cov matrix 
print("lambdas, vs =\n",lambdas,"\n",vs)

# Re-plot our data
fig = go.Figure(data=[go.Scatter3d(x=x, y=y, z=z,mode='markers',  
        marker=dict(size=8,opacity=0.5), name="data" ) ])

# Draw some extra 'lines' showing eigenvector directions
n_ev_balls = 50    # the lines will be made of lots of balls in a line
ev_size= 3    # size of balls
t = np.linspace(0,1,num=n_ev_balls)  # parameterizer for drawing along vec directions

for i in range(3):   # do this for each eigenvector 
    # Uncomment the next line to scale (unit) vector by size of the eigenvalue
    # vs[:,i] *= lambdas[i] 
    ex, ey, ez = t*vs[0,i], t*vs[1,i],  t*vs[2,i]
    fig.add_trace(go.Scatter3d(x=ex, y=ey, z=ez,mode='markers',
                marker=dict(size=ev_size,opacity=0.8), name="v_"+str(i+1)))

fig.update_layout( xaxis_title="x", yaxis_title="y", yaxis = dict(scaleanchor = "x",scaleratio = 1) )
fig.show()

In [ ]:
lambdas, vs = sorted_eig(cov)  

proj_cov = vs.T @ cov @ vs   # project the covariance matrix, using eigenvectors
proj_cov

In [ ]:
proj_cov[np.abs(proj_cov) < 1e-15] = 0
proj_cov

In [ ]:
data = np.stack((x,y,z),axis=1)
data.shape   # we had a 100 data points, so expecting 100x3 matrix 

In [ ]:
print("\n 1. All data, rotated into new coordinate system")
W = vs[:,0:3]  # keep the all the eigenvectors
new_data_all = data @ W  # project all the data 
print("Checking: new_data_all.shape =",new_data_all.shape)
print("New covariance matrix = \n",np.cov(new_data_all.T) )


print("\n 2. Truncated data projected onto principal axes of coordinate system")

W = vs[:,0:2]  # keep only the first and 2nd eigenvectors 
print("W.shape = ",W.shape) 
new_data_proj = data @ W   # project 
print("Checking: new_data_proj.shape =",new_data_proj.shape)
print("New covariance matrix in projected space = \n",np.cov(new_data_proj.T) )

# Difference between them
diff = new_data_all[:,0:2] - new_data_proj
print("\n Absolute maximum difference between the two methods = ",np.max(np.abs(diff)))

In [ ]:
fig = go.Figure(data=[(go.Scatter3d(x=new_data_all[:,0], y=new_data_all[:,1], z=new_data_all[:,2],
        mode='markers', marker=dict(size=4,opacity=0.5), name="full data" ))])
fig.add_trace(go.Scatter3d(x=new_data_proj[:,0], y=new_data_proj[:,1], z=new_data_proj[:,0]*0,
        mode='markers', marker=dict(size=4,opacity=0.5), name="projected" ) )
fig.update_layout(scene_aspectmode='data')
fig.show()

In [ ]:
from sklearn.datasets import load_digits 
from sklearn.decomposition import PCA
digits  = load_digits()
X = digits.data / 255.0 
Y = digits.target
print(X.shape, Y.shape,'\n')

# Let's look a a few examples
for i in range(8):  # show 8 examples 
    print("This is supposed to be a '",Y[i],"':",sep="")
    plt.imshow(X[i].reshape([8,8]))
    plt.show()

In [ ]:
digits_cov = np.cov(X.T)
print("digits_cov.shape = ",digits_cov.shape)
lambdas, vs = sorted_eig(np.array(digits_cov)) 

W = vs[:,0:2]  # just keep two dimensions
proj_digits = X @ W
print("proj_digits.shape = ", proj_digits.shape)

# Make the plot 
fig = go.Figure(data=[go.Scatter(x=proj_digits[:,0], y=proj_digits[:,1],# z=Y, #z=proj_digits[:,2],
                mode='markers', marker=dict(size=6, opacity=0.7, color=Y), text=['digit='+str(j) for j in Y] )])
fig.update_layout( xaxis_title="q_1", yaxis_title="q_2", yaxis = dict(scaleanchor = "x",scaleratio = 1) )
fig.update_layout(scene_camera=dict(up=dict(x=0, y=0, z=1), center=dict(x=0, y=0, z=0), eye=dict(x=0, y=0, z=1.5)))
fig.show()

In [ ]:
W = vs[:,0:3]  # just three dimensions
proj_digits = X @ W
print("proj_digits.shape = ", proj_digits.shape)

# Make the plot, separate them by "z" which is the digit of interest.  
fig = go.Figure(data=[go.Scatter3d(x=proj_digits[:,0], y=proj_digits[:,1], z=proj_digits[:,2],
                mode='markers', marker=dict(size=4, opacity=0.8, color=Y, showscale=True), 
                text=['digit='+str(j) for j in Y] )])
fig.update_layout(title="8x8 Handwritten Digits", xaxis_title="q_1", yaxis_title="q_2", yaxis = dict(scaleanchor = "x",scaleratio = 1) )
fig.show()

In [ ]:
plt.plot( np.abs(lambdas)/np.sum(lambdas) )
plt.xlabel('Number of components')
plt.ylabel('Significance') 
plt.show()

## Appendix A: Overkill: Bigger Handwritten Digits

28x28 images, as in the MNIST dataset?

http://yann.lecun.com/exdb/mnist/

In [ ]:
#from sklearn.datasets import fetch_mldata
from sklearn.datasets import fetch_openml
from random import sample 

#mnist = fetch_mldata('MNIST original')
mnist = fetch_openml('mnist_784', version=1, cache=True)

X2 = mnist.data / 255
Y2 = np.array(mnist.target,dtype=np.int) 

#Let's grab some indices for random suffling 
indices = list(range(X2.shape[0]))

# Let's look a a few examples
for i in range(8):  # 8 is good
    i = sample(indices,1)
    print("This is supposed to be a ",Y2[i][0],":",sep="")
    plt.imshow(X2[i].reshape([28,28]))
    plt.show()


In [ ]:
# Like we did before... Almost the whole PCA method is the next 3 lines! 
mnist_cov = np.cov(X2.T)
lambdas, vs = sorted_eig(np.array(mnist_cov))
W = vs[:,0:3]       # Grab the 3 most significant dimensions

# Plotting all 70,000 data points is going to be too dense too look at. 
# Instead let's grab a random sample of 5,000 points
n_plot = 5000
indices = sample(list(range(X2.shape[0])), n_plot)
proj_mnist = np.array(X2[indices] @ W, dtype=np.float32)  # Last step of PCA: project 

fig = go.Figure(data=[go.Scatter3d(x=proj_mnist[:,0], y=proj_mnist[:,1], z=proj_mnist[:,2],
                mode='markers', marker=dict(size=4, opacity=0.7, color=Y2[indices], 
                showscale=True), text=['digit='+str(j) for j in Y2[indices]] )])
fig.show()

## # Appendix B: Because We Can: Turning it into a Classifier

In [ ]:
# random shuffled ordering of the whole thing
indices = sample(list(range(X.shape[0])), X.shape[0])
X_shuf, Y_shuf = X[indices,:], Y[indices]

# 80-20 train-test split
max_train_ind = int(0.8*X.shape[0])    
X_train, Y_train = X_shuf[0:max_train_ind,:], Y_shuf[0:max_train_ind]
X_test, Y_test = X_shuf[max_train_ind:-1,:], Y_shuf[max_train_ind:-1] 

# Do PCA on training set 
train_cov = np.cov(X_train.T)
ell, v = sorted_eig(np.array(train_cov))
pca_dims = 3                # number of top 'dimensions' to take
W_train = v[:,0:pca_dims]   
proj_train = X_train @ W_train

# also project the testing set while we're at it
proj_test = X_test @ W_train   # yes, same W_train 

In [ ]:
from collections import Counter

def knn_predict(xnew, proj_train, Y_train, k=3):
    """
    xnew is a new data point that has the same shape as one row of proj_train.
    Given xnew, calculate the (squared) distance to all the points in X_train
    to find out which ones are nearest.
    """
    distances = ((proj_train - xnew)**2).sum(axis=1)  
    # stick on an extra column of indexing 'hash' for later use after we sort
    dists_i = np.stack( (distances, np.array(range(Y_train.shape[0]) )),axis=1 )
    dists_i = dists_i[dists_i[:,0].argsort()]    # sort in ascending order of distance
    knn_inds = (dists_i[0:k,-1]).astype(np.int)  # Grab the indexes for k nearest neighbors
 
    # take 'votes':
    knn_targets = list(Y_train[knn_inds])  # which classes the nn's belong to
    votes = Counter(knn_targets)         #  count up how many of each class are represented
    return votes.most_common(1)[0][0]    # pick the winner, or the first member of a tie


# Let's test it on the first element of the testing set
x, y_true = proj_test[0], Y_test[0]
guess = knn_predict(x, proj_train, Y_train)
print("guess = ",guess,", true = ",y_true)

In [ ]:
mistakes, n_test = 0, Y_test.shape[0]
for i in range(n_test):
    x = proj_test[i]
    y_pred = knn_predict(x, proj_train, Y_train, k=3)
    y_true = Y_test[i]
    if y_true != y_pred:
        mistakes += 1 
    if i < 20:   # show some examples
        print("x, y_pred, y_true =",x, y_pred, y_true, 
              "YAY!" if y_pred==y_true else "     BOO. :-(")
print("...skipping a lot...")

print("Total Accuracy =", (n_test-mistakes)/n_test*100,"%")